<a href="https://colab.research.google.com/github/Vimp17/py/blob/main/hw%5Bdone%5D_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов на основе семинарского кода. Мы будем использовать датасет [ag_news](https://paperswithcode.com/dataset/ag-news). Это датасет для классификации новостей на 4 темы: "World", "Sports", "Business", "Sci/Tech".

Установим модуль datasets, чтобы нам проще было работать с данными.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette='summer')

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных
Для вашего удобства, мы привели код обработки датасета в ноутбуке. Ваша задача --- обучить модель, которая получит максимальное возможное качество на тестовой части.

In [6]:
# Загрузим датасет
dataset = datasets.load_dataset('ag_news')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Как и в семинаре, выполним следующие шаги:
* Составим словарь
* Создадим класс WordDataset
* Выделим обучающую и тестовую часть, создадим DataLoader-ы.

In [7]:
words = Counter()

for example in tqdm(dataset['train']['text']):
    # Приводим к нижнему регистру и убираем пунктуацию
    prccessed_text = example.lower().translate(
        str.maketrans('', '', string.punctuation))

    for word in word_tokenize(prccessed_text):
        words[word] += 1


vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
counter_threshold = 25

for char, cnt in words.items():
    if cnt > counter_threshold:
        vocab.add(char)

print(f'Размер словаря: {len(vocab)}')

word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

  0%|          | 0/120000 [00:00<?, ?it/s]

Размер словаря: 11842


In [8]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        processed_text = self.data[idx]['text'].lower().translate(
            str.maketrans('', '', string.punctuation))
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) for word in word_tokenize(processed_text)
            ]
        tokenized_sentence += [self.eos_id]

        train_sample = {
            "text": tokenized_sentence,
            "label": self.data[idx]['label']
        }

        return train_sample

    def __len__(self) -> int:
        return len(self.data)


def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>'], max_len=256) -> torch.Tensor:
    seq_lens = [len(x['text']) for x in input_batch]
    max_seq_len = min(max(seq_lens), max_len)

    new_batch = []
    for sequence in input_batch:
        sequence['text'] = sequence['text'][:max_seq_len]
        for _ in range(max_seq_len - len(sequence['text'])):
            sequence['text'].append(pad_id)

        new_batch.append(sequence['text'])

    sequences = torch.LongTensor(new_batch).to(device)
    labels = torch.LongTensor([x['label'] for x in input_batch]).to(device)

    new_batch = {
        'input_ids': sequences,
        'label': labels
    }

    return new_batch

In [9]:
train_dataset = WordDataset(dataset['train'])

np.random.seed(42)
idx = np.random.choice(np.arange(len(dataset['test'])), 5000)
eval_dataset = WordDataset(dataset['test'].select(idx))

batch_size = 32
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

In [10]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
        ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0] #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1) #[batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        output = self.dropout(self.linear(self.non_lin(output)))  # [batch_size, hidden_dim]
        prediction = self.projection(self.non_lin(output))  # [batch_size, num_classes]

        return prediction

## Постановка задачи
Ваша задача -- получить максимальное возможное accuracy на `eval_dataloader`. Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы вычислить качество её работы.

In [11]:
def evaluate(model, eval_dataloader) -> float:
    """
    Calculate accuracy on validation dataloader.
    """

    predictions = []
    target = []
    with torch.no_grad():
        for batch in eval_dataloader:
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))
            target.append(batch['label'])

    predictions = torch.cat(predictions)
    target = torch.cat(target)
    accuracy = (predictions == target).float().mean().item()

    return accuracy

In [12]:
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

## Ход работы
Оценка за домашнее задание складывается из четырех частей:
### Запуск базовой модели с семинара на новом датасете (1 балл)
На семинаре мы создали модель, которая дает на нашей задаче довольно высокое качество. Ваша цель --- обучить ее и вычислить `score`, который затем можно будет использовать в качестве бейзлайна.

В модели появится одно важное изменение: количество классов теперь равно не 2, а 4. Обратите на это внимание и найдите, что в коде создания модели нужно модифицировать, чтобы учесть это различие.

### Проведение экспериментов по улучшению модели (2 балла за каждый эксперимент)
Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Каждый выполненный эксперимент будет оцениваться в 2 балла. Для получения полного балла за этот пункт вам необходимо выполнить по крайней мере 2 эксперимента. Не расстраивайтесь, если какой-то эксперимент не дал вам прироста к качеству: он все равно зачтется, если выполнен корректно.

Вот несколько идей экспериментов:
* **Модель RNN**. Попробуйте другие нейросетевые модели --- LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение архитектуры после применения RNN**. В базовой модели используется агрегация со всех эмбеддингов. Возможно, вы захотите конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Возможно, для получения более высокого качества просто необходимо увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

Обратите внимание, что главное правило проведения экспериментов --- необходимо совершать одно архитектурное изменение в одном эксперименте. Если вы совершите несколько изменений, то будет неясно, какое именно из изменений дало прирост к качеству.

### Получение высокого качества (3 балла)
В конце вашей работы вы должны указать, какая из моделей дала лучший результат, и вывести качество, которое дает лучшая модель, с помощью функции `evaluate`. Ваша модель будет оцениваться по метрике `accuracy` следующим образом:
* $accuracy < 0.9$ --- 0 баллов;
* $0.9 \leqslant accuracy < 0.91$ --- 1 балл;
* $0.91 \leqslant accuracy < 0.915$ --- 2 балла;
* $0.915 \leqslant accuracy$ --- 3 балла.

### Оформление отчета (2 балла)
В конце работы подробно опишите все проведенные эксперименты.
* Укажите, какие из экспериментов принесли улучшение, а какие --- нет.
* Проанализируйте графики сходимости моделей в проведенных экспериментах. Являются ли колебания качества обученных моделей существенными в зависимости от эпохи обучения, или же сходимость стабильная?
* Укажите, какая модель получилась оптимальной.

Желаем удачи!

In [15]:
num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy = evaluate(model, eval_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses:.4f}, Eval Accuracy: {eval_accuracy:.4f}")


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.2364, Eval Accuracy: 0.8828


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.2130, Eval Accuracy: 0.9022


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.1990, Eval Accuracy: 0.9006


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.1835, Eval Accuracy: 0.9002


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.1713, Eval Accuracy: 0.9006


In [20]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, dropout_rate=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        _, hidden = self.gru(embedded)
        hidden = self.dropout(hidden.squeeze(0))
        logits = self.fc(hidden)
        return logits

model = GRUModel(vocab_size=len(vocab),hidden_dim=256, output_dim=4).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

In [21]:
num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy = evaluate(model, eval_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.4146, Eval Accuracy: 0.8968


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.2158, Eval Accuracy: 0.9096


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.1536, Eval Accuracy: 0.9088


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.1084, Eval Accuracy: 0.9048


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.0805, Eval Accuracy: 0.9030


In [47]:
class StackedRNNModel(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int, num_classes: int = 4,
        aggregation_type: str = 'max'
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn1 = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.rnn2 = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, num_classes)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output1, _ = self.rnn1(embeddings)  # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output1 = output1.max(dim=1)[0]  # [batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output1 = output1.mean(dim=1)  # [batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type")

        # Вызов второго RNN
        output2, _ = self.rnn2(output1.unsqueeze(1))  # Добавляем размерность seq_len=1 для RNN
        output2 = output2.squeeze(1)  # Убираем размерность seq_len

        output2 = self.dropout(self.linear(self.non_lin(output2)))
        prediction = self.projection(self.non_lin(output2))  # [batch_size, num_classes]

        return prediction




model = StackedRNNModel(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())


In [48]:
num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy = evaluate(model, eval_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.4149, Eval Accuracy: 0.8984


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.2850, Eval Accuracy: 0.9040


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.2495, Eval Accuracy: 0.9018


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.2208, Eval Accuracy: 0.9066


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.2016, Eval Accuracy: 0.9034


In [49]:
class EnhancedRNNModel(nn.Module):
    def __init__(
        self,
        hidden_dim: int,
        vocab_size: int,
        output_dim: int = 4,
        dropout_rate: float = 0.3,
        aggregation_type: str = 'mean'
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=word2ind['<pad>'])
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.aggregation_type = aggregation_type

    def forward(self, input_ids):
        # Эмбеддинги входных данных
        embeddings = self.embedding(input_ids)  # [batch_size, seq_len, hidden_dim]

        # RNN слой
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]

        # Агрегация по последовательности
        if self.aggregation_type == 'mean':
            aggregated = output.mean(dim=1)  # [batch_size, hidden_dim]
        elif self.aggregation_type == 'max':
            aggregated = output.max(dim=1)[0]  # [batch_size, hidden_dim]
        else:
            raise ValueError("Invalid aggregation_type: choose 'mean' or 'max'.")

        # Последний токен
        last_token = output[:, -1, :]  # [batch_size, hidden_dim]

        # Комбинируем последние токены и агрегацию
        combined = torch.cat([last_token, aggregated], dim=1)  # [batch_size, hidden_dim * 2]

        # Dropout и финальный слой
        combined = self.dropout(combined)
        logits = self.fc(combined)  # [batch_size, output_dim]

        return logits

model = EnhancedRNNModel(hidden_dim=256, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

In [50]:
num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy = evaluate(model, eval_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.4424, Eval Accuracy: 0.8942


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.2760, Eval Accuracy: 0.9012


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.2350, Eval Accuracy: 0.8994


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.2080, Eval Accuracy: 0.9016


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.1898, Eval Accuracy: 0.8988


In [61]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, dropout_rate=0.15):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        _, hidden = self.gru(embedded)
        hidden = self.dropout(hidden.squeeze(0))
        logits = self.fc(hidden)
        return logits

model = GRUModel(vocab_size=len(vocab),hidden_dim=256, output_dim=4).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

In [69]:
def train_model(model, criterion, optimizer):
  num_epochs = 5

  eval_accuracy = []
  for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy.append(evaluate(model, eval_dataloader))
  return min(eval_accuracy)

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'hidden_dim': [128, 256, 512],
    'learning_rate': [0.001, 0.005, 0.01],

}
# Подбор параметров
best_params = None
best_loss = float('inf')

for params in ParameterGrid(param_grid):
    print(f"Пробуем параметры: {params}")
    model = GRUModel(vocab_size=len(vocab),hidden_dim=params['hidden_dim'], output_dim=4).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Разделение данных и обучение
    train_loss = train_model(model, criterion, optimizer)
    print(f"Train Loss: {train_loss}")

    # Обновление лучших параметров
    if train_loss < best_loss:
        best_loss = train_loss
        best_params = params

print(f"Лучшие параметры: {best_params}, с Loss: {best_loss}")

Пробуем параметры: {'hidden_dim': 128, 'learning_rate': 0.001}


  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

...

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Train Loss: 0.9073999524116516
Пробуем параметры: {'hidden_dim': 256, 'learning_rate': 0.005}


  0%|          | 0/3750 [00:00<?, ?it/s]

In [62]:
num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    losses = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        logits = model(batch['input_ids'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    losses = (sum(epoch_losses) / len(epoch_losses))
    eval_accuracy = evaluate(model, eval_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses:.4f}, Eval Accuracy: {eval_accuracy:.4f}")

  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.4052, Eval Accuracy: 0.9038


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.2074, Eval Accuracy: 0.9138


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.1426, Eval Accuracy: 0.9088


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.0968, Eval Accuracy: 0.9050


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.0685, Eval Accuracy: 0.9076
